In [1]:
# Imports
import os, sys
import pandas as pd

In [ ]:
"""
    Local Dataset Importing
        Originally tried to use the Kaggle API however quickly faced
        issues with rate limiting. Local downloading will speed up
        training at the cost of repeatability for others viewing project.
"""
news_path = os.getcwd() + "/../news_data/"
stock_path = os.getcwd() + "/../stock_data/"

news_df = pd.read_json(f'{news_path}News_Category_Dataset_v3.json', lines=True)
stock_meta = pd.read_csv(f'{stock_path}symbols_valid_meta.csv')

print(news_df.columns)
print(stock_meta.columns)

Index(['Nasdaq Traded', 'Symbol', 'Security Name', 'Listing Exchange',
       'Market Category', 'ETF', 'Round Lot Size', 'Test Issue',
       'Financial Status', 'CQS Symbol', 'NASDAQ Symbol', 'NextShares'],
      dtype='object')
